In [ ]:
import math
import numpy as np

class DiceGame:
    def __init__(self, target_toss=1, consecutive_needed=3, prize_money=100000,
                 failed_toss_cost=1000, success_then_failed_cost=7800,
                 two_success_then_failed_cost=49500, num_dice=1):
        self.target_toss = target_toss
        self.consecutive_needed = consecutive_needed
        self.prize_money = prize_money
        self.failed_toss_cost = failed_toss_cost
        self.success_then_failed_cost = success_then_failed_cost
        self.two_success_then_failed_cost = two_success_then_failed_cost
        self.num_dice = num_dice

    def probability_of_success(self):
        total_faces = 6
        target_faces = 1 if self.num_dice == 1 else len(
            set(str(self.num_dice * self.target_toss).zfill(2))
        )
        return target_faces / (total_faces ** self.num_dice)

    def tosses_for_confidence(self, confidence=0.95):
        p_success = self.probability_of_success()
        return math.ceil(math.log(1 - confidence) / math.log(1 - p_success ** self.consecutive_needed))

    def detailed_expected_value(self):
        """
        Calculate detailed expected value considering multiple failure scenarios
        """
        p_success = self.probability_of_success()
        p_fail = 1 - p_success

        # Probability of different consecutive failure scenarios
        p_success_then_fail = p_fail * p_success
        p_two_success_then_fail = p_fail * (p_success ** 2)

        # Expected losses from different scenarios
        loss_failed_toss = p_fail * self.failed_toss_cost
        loss_success_then_failed = p_success_then_fail * self.success_then_failed_cost
        loss_two_success_then_failed = p_two_success_then_fail * self.two_success_then_failed_cost
        expected_failure_cost = loss_failed_toss + loss_success_then_failed + loss_two_success_then_failed

        # Probability of winning
        p_win = p_success ** self.consecutive_needed
        expected_gain = p_win * self.prize_money

        # Net gain/loss per toss
        net_gain_or_loss_per_toss = expected_gain - expected_failure_cost

        return {
            "Expected Failure Cost": expected_failure_cost,
            "Net Gain/Loss per Toss": net_gain_or_loss_per_toss
        }

    def relevant_premium(self):
        expected_value = self.detailed_expected_value()
        expected_failure_cost = expected_value["Expected Failure Cost"]
        p_success = self.probability_of_success()
        p_win = p_success ** self.consecutive_needed
        expected_gain_from_winning = p_win * self.prize_money

        # Calculate the premium required for the game to be worth playing
        premium_needed = expected_failure_cost - expected_gain_from_winning
        return premium_needed

    def optimal_strategy(self, num_players=4):
        individual_prob = self.probability_of_success()
        collective_prob = 1 - (1 - individual_prob) ** num_players

        return {
            "Individual Success Probability": individual_prob,
            "Collective Success Probability": collective_prob,
        }



In [ ]:
# Game scenarios
scenarios = [
    {"name": "Single Die Standard", "num_dice": 1, "prize_money": 100000},
    {"name": "Two Dice High Stakes", "num_dice": 2, "prize_money": 3000000}
]

for scenario in scenarios:
    game = DiceGame(num_dice=scenario["num_dice"], prize_money=scenario["prize_money"])
    print(f"\n{scenario['name']} Game Analysis:")

    # 1. How many dice tosses do you need to get to 95% win confidence?
    tosses_needed = game.tosses_for_confidence(confidence=0.95)
    print(f"Tosses for 95% Confidence: {tosses_needed}")

    # 2. Calculate your net gain/loss per toss.
    expected_value = game.detailed_expected_value()
    print(f"Net Gain/Loss per Toss: {expected_value['Net Gain/Loss per Toss']}")

    # 3. What is the minimum amount of prize money you will consider playing this game for? Calculate your relevant premium in this case.
    premium_needed = game.relevant_premium()
    print(f"Premium Needed: {premium_needed}")

    # 4. Please rationalize the most efficient strategy to maximize gain/minimize loss with 4 players playing this game simultaneously.
    optimal_strategy = game.optimal_strategy(num_players=4)
    print(f"Optimal Strategy: {optimal_strategy}")


Single Die Standard Game Analysis:
Tosses for 95% Confidence: 646
Net Gain/Loss per Toss: -2599.537037037037
Premium Needed: 2599.537037037037
Optimal Strategy: {'Individual Success Probability': 0.16666666666666666, 'Collective Success Probability': 0.5177469135802468}

Two Dice High Stakes Game Analysis:
Tosses for 95% Confidence: 17470
Net Gain/Loss per Toss: -983.590534979424
Premium Needed: 983.590534979424
Optimal Strategy: {'Individual Success Probability': 0.05555555555555555, 'Collective Success Probability': 0.20438004877305294}


4.
With 4 players, the overall probability of winning the group is about 48.2%, which means at least one of the 4 players will win in a turn.
Players should coordinate and roll the dice evenly to maximize the group's chances of winning, increasing the overall probability of winning by nearly 50%.

5. In a single toss, the probability of rolling a 1 with one die is 1/6.
However, with two dice, the probability of rolling "11", "16", or "66" is 1/12. => Therefore, with one die, the success rate is higher compared to when using two dice. Furthermore, with a prize of 3,000,000, which is more than three times the cost of destroying one die, I conclude that I would decide to pay 1,000,000 to destroy one die in the final toss.